In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

In [21]:
# n_features = 16000
# n_samples = 15000
n_features = 3
n_samples = 10
n_classes = 2
X = np.random.randn(n_features, n_samples)
y = np.random.randint(0, n_classes, n_samples)
y = np.array([1 if i == 1 else -1 for i in y])
weights = np.random.randn(n_samples)


# Their method

In [59]:
s_t = time.time()

total_pos, total_neg = 0, 0
for w, label in zip(weights, y):
    if label == 1:
        total_pos += w
    else:
        total_neg += w

classifiers = []
total_features = X.shape[0]
for index, feature in enumerate(X):
    if len(classifiers) % 1000 == 0 and len(classifiers) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers), total_features))

    applied_feature = sorted(zip(weights, feature, y), key=lambda x: x[1])

    pos_seen, neg_seen = 0, 0
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
    current_idx = 0
    ws = []
    last_error = 0
    pos_seen_list = []
    for w, f, label in applied_feature:
        ws.append(w)
        # min(all before current example are positive and all after are negative, all before current example are negative and all after are positive)
        # error = sum of weights of misclassified examples
        # print("error1 : ", neg_weights + total_pos - pos_weights, ", error2 : ", pos_weights + total_neg - neg_weights, end=',  ')
        error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
        last_error = error
        # print("error : ", error)
        if error < min_error:
            min_error = error
            # best_feature = features[index]
            best_feature = (current_idx, f)
            best_threshold = f
            best_polarity = 1 if pos_seen > neg_seen else -1

        if label == 1:
            pos_seen += 1
            pos_weights += w
        else:
            neg_seen += 1
            neg_weights += w
        current_idx += 1
        pos_seen_list.append(pos_seen)
    # print("pos_seen_list : ", list(enumerate(pos_seen_list)))
    # print('best_feature[0]: ', best_feature[0])
    # print('last error:   ', last_error, 'min error:  ', min_error)

    # clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity)
    clf = best_feature, best_threshold, best_polarity, min_error
    classifiers.append(clf)

print("Time taken: %f seconds" % (time.time() - s_t))

pos_seen_list :  [(0, 0), (1, 0), (2, 1), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 3), (9, 3)]
best_feature[0]:  7
last error:    -1.3954637033063257 min error:   -3.0371732568983996
pos_seen_list :  [(0, 0), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 3), (8, 3), (9, 3)]
best_feature[0]:  2
last error:    -0.5071501435293333 min error:   -1.947853965597011
pos_seen_list :  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 1), (6, 1), (7, 1), (8, 2), (9, 3)]
best_feature[0]:  9
last error:    -1.4460947254201173 min error:   -1.4460947254201173
Time taken: 0.002145 seconds


# Mine

<h3>
<div style='color: red;'>
Question: for the polarity:<br>
 Direction that gives minimum weights <span style='color:pink'> -> To be consistent with finding minimum error using weights</span><br> 
    or<br>
 Direction that gives less misclassified samples???? <span style='color:pink'> -> To find threshold giving better accuracy???</span></div>
</h3>

In [52]:
s_t = time.time()
classifiers2 = []
total_features2 = X.shape[0]
# TODO parallize this
# TODO can we get rid of this loop and make them matrices?
for index, feature in enumerate(X):
    if len(classifiers2) % 1000 == 0 and len(classifiers2) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers2), total_features2))
        
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None

    # sort by feature value first, then by weight, then by label
    sorting_indecies = np.lexsort((y, weights, feature))
    
    # s_w: srorted weights, s_f: sorted features, s_y: sorted labels
    s_w, s_f, s_y = weights[sorting_indecies], feature[sorting_indecies], y[sorting_indecies]
    # print(s_w)
    ## left = -1, right = 1
    # negative weights
    neg_weights = np.concatenate(([0], np.cumsum(s_w * (s_y == -1))))
    # positive weights (reversed)
    pos_weights = np.flip(np.cumsum(np.flip(s_w * (s_y == 1))))
    pos_weights2 = np.concatenate((pos_weights, [0]))
    
    idx1 = np.argmin(neg_weights + pos_weights2)
    if idx1 > len(neg_weights) - 1:
        idx1 = len(neg_weights) - 1
    
    min_error1 = neg_weights[idx1] + pos_weights2[idx1]

    best_feature1, best_threshold1 = (idx1, s_f[idx1]), s_f[idx1]

    ## left = 1, right = -1
    # repeat for s_y ==1, s_y == 1
    pos_weights_rl = np.concatenate(([0], np.cumsum(s_w * (s_y == 1))))

    neg_weights_rl = np.flip(np.cumsum(np.flip(s_w * (s_y == -1))))
    neg_weights_rl = np.concatenate((neg_weights_rl, [0]))
    idx2 = np.argmin(neg_weights_rl + pos_weights_rl)
    if idx2 > len(neg_weights_rl) - 1:
        idx2 = len(neg_weights_rl) - 1
    
    min_error2 = neg_weights_rl[idx2] + pos_weights_rl[idx2]

    best_feature2, best_threshold2 = (idx2, s_f[idx2]), s_f[idx2]


    # print("error1 : ", min_error1, ", error2 : ", min_error2, end=',  ')
    # print("error : ", min(min_error1, min_error2))
    if min_error1 < min_error2:
        min_error = min_error1
        best_feature = best_feature1
        best_threshold = best_threshold1
        #@ wrong, should be number of true predictions, not weights??
        best_polarity = -1
    else:
        min_error = min_error2
        best_feature = best_feature2
        best_threshold = best_threshold2
        best_polarity = 1

    clf2 = best_feature, best_threshold, best_polarity, min_error
    classifiers2.append(clf2)

    

print("Time taken: %f seconds" % (time.time() - s_t))

error1 :  -3.0371732568983996 , error2 :  -1.5530697218973466,  error :  -3.0371732568983996
error1 :  -1.7547921911517586 , error2 :  -1.947853965597011,  error :  -1.947853965597011
error1 :  -1.4460947254201173 , error2 :  -1.0980456496607842,  error :  -1.4460947254201173
Time taken: 0.001290 seconds


In [53]:
for el in classifiers:
    print(el)
print()
for el in classifiers2:
    print(el)
classifiers==classifiers2
# print(classifiers[0])
# print(classifiers2[0])

((7, 0.4388840700609143), 0.4388840700609143, -1, -3.0371732568983996)
((2, 0.03704944486883128), 0.03704944486883128, -1, -1.947853965597011)
((9, 1.588808849994089), 1.588808849994089, -1, -1.4460947254201173)

((7, 0.4388840700609143), 0.4388840700609143, -1, -3.0371732568983996)
((2, 0.03704944486883128), 0.03704944486883128, 1, -1.947853965597011)
((9, 1.588808849994089), 1.588808849994089, -1, -1.4460947254201173)


False

In [42]:
a1 = np.zeros((len(classifiers), 5))
for i in range(len(classifiers)):
    a1[i] = np.array([*classifiers[i][0], classifiers[i][1], classifiers[i][2], classifiers[i][3]])

a2 = np.zeros((len(classifiers2), 5))
for i in range(len(classifiers2)):
    a2[i] = np.array([*classifiers2[i][0], classifiers2[i][1], classifiers2[i][2], classifiers2[i][3]])

classifiers
print(a1[a1!=a2])
print(a2[a1!=a2])

[-1.]
[1.]
